In [2]:
## This file implements neural networks and logistic regression on p003ppresabs_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## To deal with overfitting problem, we include both dropout and regularizer when set up layers in neural networks.
## For fully-connected neural networks, the accuracy is 97.01% for combination data, and 97.92% for over-sampling data.
## For logistic regression, the accuracy is 100% for combination data, and 97.92% for over-sampling data.
## Since the accuracy scores are pretty high in logistic regression, we further construct random forest models, which 
## are relatively less likely to bring overfitting compared to decision tree.
## For random forest, the accuracy is 99.25% for combination data, and 100% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.41% for combination data, and 97.63% for over-sampling.

In [3]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [4]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p003ppresabs_quant.csv')
df.shape

(255, 866)

In [5]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [6]:
df['pheno']

0      0.169500
1      0.316500
2      0.460000
3      0.169000
4      0.171333
5      0.236333
6      0.166500
7      0.173333
8      0.315833
9      0.162167
10     0.186167
11     0.382833
12     0.170667
13     0.173333
14     0.162833
15     0.164333
16     0.382667
17     0.397000
18     0.372667
19     0.166333
20     0.242333
21     0.305833
22     0.162333
23     0.165500
24     0.616833
25     0.170167
26     0.491500
27     0.167833
28     0.166500
29     0.374833
         ...   
225    0.267200
226    0.293800
227    0.383400
228    0.513500
229    0.507700
230    0.311500
231    0.236000
232    0.228833
233    0.219333
234    0.167333
235    0.265167
236    0.160500
237    0.159333
238    0.165500
239    0.327833
240    0.342000
241    0.308500
242    0.170333
243    0.165333
244    0.340833
245    0.162000
246    0.170500
247    0.164500
248    0.164000
249    0.180000
250    0.348000
251    0.232500
252    0.166167
253    0.276667
254    0.225333
Name: pheno, Length: 255

In [7]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [8]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [9]:
df['pheno']
df['pheno'].value_counts()

0    240
1     15
Name: pheno, dtype: int64

In [10]:
df.shape

(255, 866)

In [11]:
df_clean = df.drop(columns=['id'])

In [12]:
df_clean.shape

(255, 865)

In [13]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 864) (255,)


In [14]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 209), (1, 235)]


In [15]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 240), (1, 240)]


In [16]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [17]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [18]:
############# Fully-Connected Neural Network ################

In [19]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1

In [20]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [21]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 310 samples, validate on 134 samples
Epoch 1/100
310/310 [==============================] - 0s 986us/step - loss: 2.5191 - accuracy: 0.6387 - val_loss: 0.6907 - val_accuracy: 0.7687
Epoch 2/100
310/310 [==============================] - 0s 154us/step - loss: 2.4748 - accuracy: 0.7774 - val_loss: 0.5912 - val_accuracy: 0.8209
Epoch 3/100
310/310 [==============================] - 0s 143us/step - loss: 2.0341 - accuracy: 0.7903 - val_loss: 0.5504 - val_accuracy: 0.8955
Epoch 4/100
310/310 [==============================] - 0s 141us/step - loss: 2.2436 - accuracy: 0.7871 - val_loss: 0.5483 - val_accuracy: 0.8806
Epoch 5/100
310/310 [==============================] - 0s 139us/step - loss: 1.7846 - accuracy: 0.8226 - val_loss: 0.5158 - val_accuracy: 0.8881
Epoch 6/100
310/310 [==============================] - 0s 168us/step - loss: 2.2526 - accuracy: 0.8000 - val_loss: 0.4986 - val_accuracy: 0.9030
Epoch 7/100
310/310 [==============================] - 0s 171us/step - loss: 2.1586 

Epoch 57/100
310/310 [==============================] - 0s 137us/step - loss: 1.5642 - accuracy: 0.9065 - val_loss: 0.3026 - val_accuracy: 0.9776
Epoch 58/100
310/310 [==============================] - 0s 110us/step - loss: 2.5020 - accuracy: 0.8484 - val_loss: 0.3070 - val_accuracy: 0.9701
Epoch 59/100
310/310 [==============================] - 0s 108us/step - loss: 1.6561 - accuracy: 0.9000 - val_loss: 0.2994 - val_accuracy: 0.9776
Epoch 60/100
310/310 [==============================] - 0s 82us/step - loss: 1.8046 - accuracy: 0.8903 - val_loss: 0.2993 - val_accuracy: 0.9776
Epoch 61/100
310/310 [==============================] - 0s 104us/step - loss: 1.3584 - accuracy: 0.9226 - val_loss: 0.2996 - val_accuracy: 0.9776
Epoch 62/100
310/310 [==============================] - 0s 99us/step - loss: 1.6017 - accuracy: 0.9065 - val_loss: 0.2958 - val_accuracy: 0.9776
Epoch 63/100
310/310 [==============================] - 0s 108us/step - loss: 1.7536 - accuracy: 0.8968 - val_loss: 0.2990 - v

In [23]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

134/134 [==============================] - 0s 68us/step
combination test accuracy: 97.01%


In [24]:
###### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [25]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [26]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 336 samples, validate on 144 samples
Epoch 1/100
336/336 [==============================] - 0s 705us/step - loss: 2.7142 - accuracy: 0.6339 - val_loss: 0.7194 - val_accuracy: 0.7222
Epoch 2/100
336/336 [==============================] - 0s 96us/step - loss: 2.1648 - accuracy: 0.7292 - val_loss: 0.6373 - val_accuracy: 0.7847
Epoch 3/100
336/336 [==============================] - 0s 99us/step - loss: 2.1670 - accuracy: 0.7560 - val_loss: 0.6098 - val_accuracy: 0.7847
Epoch 4/100
336/336 [==============================] - 0s 99us/step - loss: 1.9750 - accuracy: 0.7500 - val_loss: 0.5725 - val_accuracy: 0.8264
Epoch 5/100
336/336 [==============================] - 0s 99us/step - loss: 2.5743 - accuracy: 0.7440 - val_loss: 0.5553 - val_accuracy: 0.8958
Epoch 6/100
336/336 [==============================] - 0s 98us/step - loss: 1.7486 - accuracy: 0.8185 - val_loss: 0.5423 - val_accuracy: 0.8958
Epoch 7/100
336/336 [==============================] - 0s 107us/step - loss: 1.9235 - acc

336/336 [==============================] - 0s 82us/step - loss: 1.5709 - accuracy: 0.8929 - val_loss: 0.3109 - val_accuracy: 0.9722
Epoch 58/100
336/336 [==============================] - 0s 77us/step - loss: 2.3852 - accuracy: 0.8363 - val_loss: 0.3139 - val_accuracy: 0.9861
Epoch 59/100
336/336 [==============================] - 0s 75us/step - loss: 2.0401 - accuracy: 0.8571 - val_loss: 0.3139 - val_accuracy: 0.9861
Epoch 60/100
336/336 [==============================] - 0s 76us/step - loss: 1.8501 - accuracy: 0.8661 - val_loss: 0.3258 - val_accuracy: 0.9861
Epoch 61/100
336/336 [==============================] - 0s 75us/step - loss: 1.7535 - accuracy: 0.8780 - val_loss: 0.3064 - val_accuracy: 0.9861
Epoch 62/100
336/336 [==============================] - 0s 78us/step - loss: 1.3445 - accuracy: 0.8958 - val_loss: 0.3014 - val_accuracy: 0.9792
Epoch 63/100
336/336 [==============================] - 0s 76us/step - loss: 1.8949 - accuracy: 0.8661 - val_loss: 0.3061 - val_accuracy: 0.972

In [27]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

144/144 [==============================] - 0s 66us/step
over-sampling test accuracy: 97.92%


In [28]:
############ Logistic Regression ############

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [30]:
###### logistics on combination data
log_comb = LogisticRegression(random_state=123)
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 100.00%


In [32]:
##### logistics on over-sampling data
log_over = LogisticRegression(random_state=123)
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=123, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 97.92%


In [34]:
############## Random Forest ##############

In [35]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100, random_state=123)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [36]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 99.25%


In [37]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100, random_state=123)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=123,
                       verbose=0, warm_start=False)

In [38]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 100.00%


In [39]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [40]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rf_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.92063492 1.         1.         1.         1.        ]
0.9841269841269842


In [42]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rf_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.97058824 0.95588235 0.98529412 1.         0.96969697]
0.9762923351158646
